In [1]:
# ==========================================
# Median Elimination Algorithm and Utilities
# ==========================================

import numpy as np
from math import log, ceil
from collections import defaultdict

def median_elimination(arm_pulls, epsilon=0.1, delta=0.05):
    """
    Median Elimination (PAC-style) for best-arm identification.

    Args:
        arm_pulls (dict): {arm_index: [(reward, cost), ...]}
        epsilon (float): Accuracy parameter (ε-optimal arm).
        delta (float): Confidence parameter (1 - δ success).

    Returns:
        chosen_arm (int): Selected ε-optimal arm.
        total_pulls (int): Total number of pulls used.
        total_cost (float): Total cost incurred.
        confidence (float): Lower bound on probability of correctness.
        stopping_early (bool): True if terminated due to data exhaustion.
    """
    S = list(arm_pulls.keys())
    ε_l = epsilon / 4
    δ_l = delta / 2
    total_pulls = 0
    total_cost = 0
    sample_means = {}
    num_pulls = defaultdict(int)
    stopping_early = False

    while len(S) > 1:
        t_l = ceil((4 / (ε_l ** 2)) * log(3 / δ_l))

        max_available = min([len(arm_pulls[arm]) - num_pulls[arm] for arm in S])
        if max_available <= 0:
            stopping_early = True
            break
        t_l = min(t_l, max_available)

        empirical_means = {}
        for arm in S:
            start = num_pulls[arm]
            pulls = arm_pulls[arm][start:start + t_l]
            rewards = [r for r, _ in pulls]
            costs = [c for _, c in pulls]

            if not rewards:
                continue

            avg_reward = np.mean(rewards)
            empirical_means[arm] = avg_reward
            sample_means[arm] = avg_reward
            num_pulls[arm] += len(rewards)
            total_pulls += len(rewards)
            total_cost += sum(costs)

        if not empirical_means:
            stopping_early = True
            break

        median_value = np.median(list(empirical_means.values()))
        S = [arm for arm in S if empirical_means.get(arm, 0) >= median_value]

        ε_l *= 0.75
        δ_l *= 0.5

    chosen_arm = S[0] if S else None
    confidence = hoeffding_confidence(chosen_arm, sample_means, num_pulls)
    return chosen_arm, total_pulls, total_cost, confidence, stopping_early


def hoeffding_confidence(chosen_arm, sample_means, num_pulls):
    """
    Returns a conservative lower bound on the confidence that chosen_arm is ε-optimal.
    """
    chosen_mean = sample_means.get(chosen_arm, 0)
    confidences = []
    for arm, mean in sample_means.items():
        if arm == chosen_arm:
            continue
        delta = chosen_mean - mean
        if delta <= 0:
            confidences.append(0.0)
        else:
            n = min(num_pulls.get(arm, 1), num_pulls.get(chosen_arm, 1))
            conf = 1 - np.exp(-0.5 * n * delta ** 2)
            confidences.append(conf)
    return min(confidences) if confidences else 1.0
w